In [196]:
%reload_ext autoreload
%autoreload 2
import chess.pgn
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from chess_dataset import ChessDataset, ChessPairDataset
import helper

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [197]:
train_chess_pair_dataset = ChessPairDataset("dataset/npy/", portion=0.05)
test_chess_pair_dataset = ChessPairDataset("dataset/npy/", train=False, portion=0.05)
train_chess_dataset = ChessDataset("dataset/npy/")
test_chess_dataset = ChessDataset("dataset/npy/", train=False)

In [198]:
len(train_chess_pair_dataset), len(test_chess_pair_dataset)

(189728, 47432)

In [199]:
len(train_chess_dataset), len(test_chess_dataset)

(2776, 694)

In [200]:
chess_trainloader = DataLoader(train_chess_dataset, batch_size=64, shuffle=True)
chess_testloader = DataLoader(test_chess_dataset, batch_size=64, shuffle=False)
chess_pair_trainloader = DataLoader(train_chess_pair_dataset, batch_size=128, shuffle=True)
chess_pair_testloader = DataLoader(test_chess_pair_dataset, batch_size=128, shuffle=True)

In [201]:
def freeze(layer):
  layer.weight.requires_grad = False
  layer.bias.requires_grad = False

def unfreeze(layer):
  layer.weight.requires_grad = True
  layer.bias.requires_grad = True
  
# Dense encoding layers
en1 = nn.Linear(773, 600)
bne1 = nn.BatchNorm1d(600)
en2 = nn.Linear(600, 400)
bne2 = nn.BatchNorm1d(400)
en3 = nn.Linear(400, 200)
bne3 = nn.BatchNorm1d(200)
en4 = nn.Linear(200, 100)
bne4 = nn.BatchNorm1d(100)

# Dense decoding layers
de1 = nn.Linear(600, 773)
bnd1 = nn.BatchNorm1d(773)
de2 = nn.Linear(400, 600)
bnd2 = nn.BatchNorm1d(600)
de3 = nn.Linear(200, 400)
bnd3 = nn.BatchNorm1d(400)
de4 = nn.Linear(100, 200)
bnd4 = nn.BatchNorm1d(200)

# Autoencoder train
# 773 - 600 - 773
ae1 = nn.Sequential(en1, nn.ReLU(), bne1,
                    de1, nn.Sigmoid())
helper.train_autoencoder(ae1, chess_trainloader, chess_testloader, 20, 2, 0.001)

# * means freeze
# 773 *- 600 - 400 - 600 *- 773
freeze(en1)
freeze(de1)
ae2 = nn.Sequential(en1, nn.ReLU(), bne1, en2, nn.ReLU(), bne2,
                    de2, nn.ReLU(), bnd2, de1, nn.Sigmoid())
helper.train_autoencoder(ae2, chess_trainloader, chess_testloader, 20, 2, 0.001)

# 773 *- 600 *- 400 - 200 - 400 *- 600 *- 773
freeze(en2)
freeze(de2)
ae3 = nn.Sequential(en1, nn.ReLU(), bne1, en2, nn.ReLU(), bne2, en3, nn.ReLU(), bne3,
                    de3, nn.ReLU(), bnd3, de2, nn.ReLU(), bnd2, de1, nn.Sigmoid())
helper.train_autoencoder(ae3, chess_trainloader, chess_testloader, 20, 2, 0.001)

# 773 *- 600 *- 400 *- 200 - 100 - 200 *- 400 *- 600 *- 773
freeze(en3)
freeze(de3)
ae4 = nn.Sequential(en1, nn.ReLU(), bne1, en2, nn.ReLU(), bne2, en3, nn.ReLU(), bne3, en4, nn.ReLU(), bne4, 
                    de4, nn.ReLU(), bnd4, de3, nn.ReLU(), bnd3, de2, nn.ReLU(), bnd2, de1, nn.Sigmoid())
helper.train_autoencoder(ae4, chess_trainloader, chess_testloader, 20, 2, 0.001)

unfreeze(en1)
unfreeze(en1)
unfreeze(en3)
pos2vec = nn.Sequential(en1, nn.ReLU(), bne1, nn.Dropout(0.3), en2, nn.ReLU(), bne2, nn.Dropout(0.3), en3, nn.ReLU(), bne3, nn.Dropout(0.3), en4, nn.ReLU(), bne4, nn.Dropout(0.3))

start


Epoch: 0 | Loss: 0.455944: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:01<00:00, 43.54it/s]
Loss: 0.445026: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 56.42it/s]
Epoch: 1 | Loss: 0.162227: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 66.08it/s]
Loss: 0.113215: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 88.86it/s]
Epoch: 2 | Loss: 0.058064: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 64.75it/s]
Loss: 0.049382: 100%|███████████████████

end
start


Epoch: 0 | Loss: 0.364668: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 53.85it/s]
Loss: 0.368563: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 78.37it/s]
Epoch: 1 | Loss: 0.163961: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 53.12it/s]
Loss: 0.153353: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 73.21it/s]
Epoch: 2 | Loss: 0.125476: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 49.93it/s]
Loss: 0.111008: 100%|███████████████████

end
start


Epoch: 0 | Loss: 0.060639: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 44.44it/s]
Loss: 0.060367: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 67.63it/s]
Epoch: 1 | Loss: 0.044910: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 44.01it/s]
Loss: 0.051152: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 64.73it/s]
Epoch: 2 | Loss: 0.038734: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 46.22it/s]
Loss: 0.046750: 100%|███████████████████

end
start


Epoch: 0 | Loss: 0.081142: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:01<00:00, 43.03it/s]
Loss: 0.085112: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 64.99it/s]
Epoch: 1 | Loss: 0.058692: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:01<00:00, 40.84it/s]
Loss: 0.067395: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 61.67it/s]
Epoch: 2 | Loss: 0.051426: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:01<00:00, 41.44it/s]
Loss: 0.059488: 100%|███████████████████

end


In [202]:
class DeepChess(nn.Module):
  def __init__(self, pos2vec):
    super(DeepChess, self).__init__()
    self.pos2vec = pos2vec
    self.base = nn.Sequential(nn.Linear(200, 400), nn.ReLU(), nn.BatchNorm1d(400), nn.Dropout(0.5),
                              nn.Linear(400, 200), nn.ReLU(), nn.BatchNorm1d(200), nn.Dropout(0.5),
                              nn.Linear(200, 100), nn.ReLU(), nn.BatchNorm1d(100), nn.Dropout(0.5),
                              nn.Linear(100, 2), nn.Softmax(dim=1))
  
  def forward(self, input1, input2):
    first_board = self.pos2vec(input1)
    second_board = self.pos2vec(input2)
    concat = torch.cat((first_board, second_board), 1)
    return self.base(concat)

In [203]:
dp = DeepChess(pos2vec)
dp

DeepChess(
  (pos2vec): Sequential(
    (0): Linear(in_features=773, out_features=600, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=600, out_features=400, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=400, out_features=200, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=200, out_features=100, bias=True)
    (13): ReLU()
    (14): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Dropout(p=0.3, inplace=False)
  )
  (base): Sequential(
    (0): Linear(in_features=200, out_features=400, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(400, 

In [204]:
helper.train_supervise(model=dp, train_data=chess_pair_trainloader, val_data=chess_pair_testloader, epochs=10, patient=1, lr=0.001)

Epoch: 0 | Loss: 0.000378 | Accuracy: 0.989264: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1483/1483 [01:15<00:00, 19.66it/s]
Loss: 1.152581 | Accuracy: 0.744961: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 371/371 [00:15<00:00, 24.35it/s]
Epoch: 1 | Loss: 0.000252 | Accuracy: 0.999220: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1483/1483 [01:16<00:00, 19.29it/s]
Loss: 2.213654 | Accuracy: 0.710280: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 371/371 [00:15<00:00, 24.47it/s]
Epoch: 2 | Loss: 0.000239 | Accuracy: 0.999262: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1483/1483 [01:14<00:00, 19.80it/s]
Loss: 2.040092 | Accuracy: 0.763851: 100

Early stopping


[tensor(1.1526, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(2.2137, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(2.0401, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(2.3610, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(1.7573, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(2.5191, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(1.8481, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(2.2020, grad_fn=<BinaryCrossEntropyBackward0>)]

In [206]:
i = next(iter(chess_pair_testloader))
z = i[2]
zz = dp(i[0], i[1])
print(torch.argmax(z, 1))
#print(zz)
print(torch.argmax(zz, 1))
# for a in zip(z, zz):
#   print(a)
print(sum(1 for x in (torch.argmax(z, 1) == torch.argmax(zz, 1)) if x), "/", i[0].shape[0])

tensor([0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
        1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
        1, 0, 0, 0, 0, 1, 0, 0])
tensor([0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
        1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 1, 0, 0])
98 / 128


In [207]:
torch.save(dp.state_dict(), "model/save/deepchess.pth")
torch.save(pos2vec.state_dict(), "model/save/pos2vec.pth")